# Space Policy Basics (Licensing, Spectrum, Debris)
## Module 4: Human Factors, Regulations, and Space Policy

This notebook explains **why policy and regulation matter to engineers**.

### What you’ll learn
- The main “policy buckets” that shape space missions
- Who does what (FAA, FCC, ITU, UNOOSA, and international equivalents)
- A tiny **toy** risk model you can tweak (for intuition)

### Important note
This is **not legal advice** — it’s an engineering‑focused overview to build intuition.

---

## 1) Why policy shows up in engineering

Even a technically perfect spacecraft can fail as a mission if:
- It can’t get a **launch / reentry license**
- It can’t legally use **radio spectrum**
- It creates unacceptable **debris risk**
- It violates **planetary protection** or safety rules

So teams treat policy like another set of constraints — like mass, power, and delta‑V.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("dark_background")

print("Environment ready. Let's look at policy constraints.")

## 2) The big policy buckets (engineering view)

Here are common categories engineers track:

- **Launch / reentry licensing**
  - Safety analysis, hazard areas, flight termination systems, environmental review
- **Spectrum (radio communications)**
  - Frequency coordination, interference avoidance, ground station licensing
- **Orbital debris mitigation**
  - End‑of‑life disposal, collision avoidance, passivation, conjunction operations
- **Remote sensing rules** (Earth observation)
  - Imaging licensing, data handling, export controls
- **Planetary protection** (Moon/Mars missions)
  - Contamination control requirements that affect spacecraft design and operations

Different countries have different agencies, but the structure is similar.


## 3) A toy “policy risk” score (for intuition)

Real compliance is not a single number, but it’s still useful to build intuition.

We’ll make a very simple score from 0–100:
- 0–30: low
- 30–60: medium
- 60–100: high

You’ll see how mission choices (orbit altitude, comms, imaging, crewed vs uncrewed) change the score.


In [ ]:
MISSION_PRESETS = {
    "LEO communications satellite": {
        "crewed": False,
        "earth_observation": False,
        "uses_spectrum": True,
        "altitude_km": 550,
        "disposal_plan": True,
    },
    "LEO Earth observation satellite": {
        "crewed": False,
        "earth_observation": True,
        "uses_spectrum": True,
        "altitude_km": 650,
        "disposal_plan": True,
    },
    "GEO communications satellite": {
        "crewed": False,
        "earth_observation": False,
        "uses_spectrum": True,
        "altitude_km": 35786,
        "disposal_plan": True,
    },
    "Crewed capsule (LEO)": {
        "crewed": True,
        "earth_observation": False,
        "uses_spectrum": True,
        "altitude_km": 400,
        "disposal_plan": True,
    },
    "Mars transfer spacecraft": {
        "crewed": False,
        "earth_observation": False,
        "uses_spectrum": True,
        "altitude_km": 0,
        "disposal_plan": False,
        "planetary_protection": True,
    },
}


def policy_risk_score(mission: dict) -> dict:
    """Return a breakdown + total score (0-100). Toy model for learning only."""
    score = 0
    breakdown = {}

    # Launch / reentry licensing tends to be more intense for crewed or high-energy missions.
    launch = 15
    if mission.get("crewed"):
        launch += 20
    if mission.get("planetary_protection"):
        launch += 10
    breakdown["launch_reentry"] = min(30, launch)

    # Spectrum always matters if you transmit.
    spectrum = 0
    if mission.get("uses_spectrum"):
        spectrum = 15
    breakdown["spectrum"] = spectrum

    # Earth observation tends to add licensing + data controls.
    eo = 0
    if mission.get("earth_observation"):
        eo = 15
    breakdown["remote_sensing"] = eo

    # Debris risk is often higher in crowded LEO and depends on disposal plan.
    alt = mission.get("altitude_km", 0)
    debris = 5
    if 300 <= alt <= 2000:  # crowded LEO
        debris += 20
    elif alt > 2000:
        debris += 10

    if not mission.get("disposal_plan", False):
        debris += 15
    breakdown["debris"] = min(35, debris)

    total = sum(breakdown.values())
    return {"breakdown": breakdown, "total": min(100, total)}


def plot_breakdown(label: str):
    result = policy_risk_score(MISSION_PRESETS[label])
    parts = result["breakdown"]

    names = list(parts.keys())
    values = np.array([parts[k] for k in names])

    fig, ax = plt.subplots(figsize=(10, 3.8))
    bars = ax.barh(names, values, color="#00d4ff", alpha=0.85)
    ax.set_xlim(0, 40)
    ax.set_title(f"Toy policy risk breakdown — {label}\nTotal score: {result['total']}/100")
    ax.grid(True, axis="x", alpha=0.25)

    for b in bars:
        ax.text(b.get_width() + 0.6, b.get_y() + b.get_height() / 2, f"{b.get_width():.0f}", va="center")

    plt.show()


plot_breakdown("LEO Earth observation satellite")

## 4) What can I do next?

- **Run the full project** (more scenarios + structured categories):

```bash
cd src/Module_04_Human_Factors/Projects/Space_Policy_Calculator
pip install -r requirements.txt
python calculator.py
```

- **Try this quick exercise:**
  - Change the preset in `plot_breakdown(...)` to:
    - `"GEO communications satellite"`
    - `"Crewed capsule (LEO)"`
  - What categories changed the most, and why?

---

References to explore:
- **UNOOSA** debris guidelines
- **ITU** spectrum coordination basics
- National licensing regulators (FAA in the US, and equivalents elsewhere)
